<a href="https://colab.research.google.com/github/nyanta012/gcp-linebot-llm-tutorial/blob/main/section4/firestore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colaboratoryのショートカットキー
1. **現在のセルを実行し、次のセルを選択**: `Shift + Enter`
2. **コードセルをテキストセルにする**: `Ctrl + M M`
3. **コードセルを上に追加**: `Ctrl + M A`
4. **コードセルを下に追加**: `Ctrl + M B`
5. **セルの削除**: `Ctrl + M D`

## 認証

In [ ]:
from google.colab import auth
auth.authenticate_user()

## Firestore APIの使用

In [ ]:
from google.cloud import firestore

### 書き込みデモ

In [ ]:
import datetime
# Firestoreへの接続
db = firestore.Client(project="tonal-limiter-398122")

In [ ]:
# ブログ記事を追加
blog_data = {
    "title": "My First Blog",
    "content": "This is my first blog post!",
    "posted_at": datetime.datetime.now(),
}
blog_ref = db.collection('blogs').add(blog_data)[1]

In [ ]:
# 2つ目のブログ記事を追加
blog_data = {
    "title": "My Second Blog",
    "content": "This is my second blog post!",
    "posted_at": datetime.datetime.now(),
}
db.collection('blogs').add(blog_data)

(DatetimeWithNanoseconds(2023, 9, 5, 23, 53, 55, 159318, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x7f9fbc7d2c80>)

In [ ]:
# コメントを追加（サブコレクションを使用）
comment_data = {
    "user": "john_doe",
    "comment": "Great post!",
    "commented_at": datetime.datetime.now(),
}
blog_ref.collection('comments').add(comment_data)

(DatetimeWithNanoseconds(2023, 9, 5, 23, 56, 8, 158186, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x7f9fbc7d2b60>)

In [ ]:
# コメントを追加（サブコレクションを使用）
comment_data = {
    "user": "steve",
    "comment": "Excellent post!",
    "commented_at": datetime.datetime.now(),
}
blog_ref.collection('comments').add(comment_data)

(DatetimeWithNanoseconds(2023, 9, 6, 0, 4, 1, 979109, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x7f9fbc7d0160>)

### 会話履歴の保存

In [ ]:
def insert_message(conversation_id: str, sender: str, receiver: str, message: str) -> None:
    # コレクション参照
    collection_ref = (
        db.collection("conversations").document(conversation_id).collection("messages")
    )

    # 挿入するデータ
    data = {
        "sender": sender,
        "receiver": receiver,
        "message": message,
        "timestamp": datetime.datetime.now(),
    }

    # データをFirestoreに挿入
    collection_ref.add(data)

In [ ]:
insert_message(conversation_id="conversation1", sender="user1", receiver="user2", message="こんにちは！")
insert_message(conversation_id="conversation1", sender="user2", receiver="user1", message="久しぶり〜")
insert_message(conversation_id="conversation1", sender="user1", receiver="user2", message="1年ぶりくらいかな")

In [ ]:
insert_message(conversation_id="conversation2", sender="user3", receiver="user4", message="明日の宿題何だっけ")
insert_message(conversation_id="conversation2", sender="user4", receiver="user3", message="何の科目の話？")
insert_message(conversation_id="conversation2", sender="user3", receiver="user4", message="物理")

### 読み込み

In [ ]:
collection_ref = db.collection("conversations").document("conversation1").collection("messages")

# コレクション内の全てのドキュメントを取得
docs = collection_ref.get()

In [ ]:
for doc in docs:
  data = doc.to_dict()
  message = {
      "sender": data["sender"],
      "receiver": data["receiver"],
      "message": data["message"],
      "timestamp": data["timestamp"],
  }
  print(message)

{'sender': 'user1', 'receiver': 'user2', 'message': '1年ぶりくらいかな', 'timestamp': DatetimeWithNanoseconds(2023, 9, 6, 0, 8, 40, 556860, tzinfo=datetime.timezone.utc)}
{'sender': 'user2', 'receiver': 'user1', 'message': '久しぶり〜', 'timestamp': DatetimeWithNanoseconds(2023, 9, 6, 0, 8, 40, 341048, tzinfo=datetime.timezone.utc)}
{'sender': 'user1', 'receiver': 'user2', 'message': 'こんにちは！', 'timestamp': DatetimeWithNanoseconds(2023, 9, 6, 0, 8, 39, 390707, tzinfo=datetime.timezone.utc)}


In [ ]:
# 最新のものだけを取得
collection_ref = db.collection("conversations").document("conversation1").collection("messages")
query = collection_ref.order_by("timestamp", direction=firestore.Query.DESCENDING).limit(1)
docs = query.get()

In [ ]:
for doc in docs:
  data = doc.to_dict()
  message = {
      "sender": data["sender"],
      "receiver": data["receiver"],
      "message": data["message"],
      "timestamp": data["timestamp"],
  }
  print(message)

{'sender': 'user1', 'receiver': 'user2', 'message': '1年ぶりくらいかな', 'timestamp': DatetimeWithNanoseconds(2023, 9, 6, 0, 8, 40, 556860, tzinfo=datetime.timezone.utc)}
